In [28]:
# Major usage of: https://gist.github.com/dannguyen/c9cb220093ee4c12b840
guardian_db = '/home/ostapkharysh/Documents/bt_data/DB/Guardian'

In [29]:
import json
import requests
from os import makedirs
from os.path import join, exists, abspath
from datetime import date, timedelta
from newspaper import Article
import pandas as pd
import datetime
from requests.exceptions import HTTPError
from db_manager import *
import re

In [30]:
def get_text(link):
    try:
        r = requests.get(link.strip())
        r.raise_for_status()
        article = Article(link.strip())
        article.download()
        article.parse()
        return article.text
    except HTTPError:
        print("No article found by this link!", article.url) 

In [31]:
#ARTICLES_DIR = join('/home/ostapkharysh/Documents/bt_data/Guardian_news', 'tempdata', 'articles')
#makedirs(ARTICLES_DIR, exist_ok=True)

In [32]:
MY_API_KEY = "0abd707a-96aa-4e21-b5cd-b314c6955753"
API_ENDPOINT = 'http://content.guardianapis.com/search'
my_params = {
    'from-date': "",
    'to-date': "",
    'order-by': "newest",
    'show-fields': 'all',
    'page-size': 200,
    'api-key': MY_API_KEY,
    'lang': 'en'
}

In [33]:
# day iteration from here:
conn = create_connection(guardian_db)

start_date = date(2013, 1, 1)  #month, day
end_date = date(2013,1, 1)
dayrange = range((end_date - start_date).days + 1)
for daycount in dayrange:
    dt = start_date + timedelta(days=daycount)
    datestr = dt.strftime('%Y-%m-%d')
    #fname = join(ARTICLES_DIR, datestr + '.json')
    print("Downloading", datestr)
    all_results = []
    my_params['from-date'] = datestr
    my_params['to-date'] = datestr
    current_page = 1
    total_pages = 1
    while current_page <= total_pages:
        print("...page", current_page)
        my_params['page'] = current_page
        resp = requests.get(API_ENDPOINT, my_params)
        data = resp.json()
        all_results.extend(data['response']['results'])
        # if more than one page
        current_page += 1
        total_pages = data['response']['pages']
        #print(total_pages)

    #with open(fname, 'w') as f:
    #print("Writing to", fname)
    try:
        df = pd.DataFrame(all_results)
        s = [[df.iloc[i]['webPublicationDate'], df.iloc[i]['webTitle'], get_text(df.iloc[i]['webUrl']),
              df.iloc[i]['webUrl']] for i in range(len(df))]
        #print(s[0])
        with conn:
            for el in s:
                news = ( datetime.datetime.strptime(el[0], "%Y-%m-%dT%H:%M:%SZ"), el[1], el[2], el[3])
                create_task(conn, news)
    except HTTPError:
        print("No article found by this link!", article.url)
        #f.write(json.dumps(all_results, indent=2))
close_db(conn)
print("FINISHED!")

...page 1
FINISHED!


In [34]:
import pandas as pd
with open('/home/ostapkharysh/Documents/bt_data/Guardian_news/tempdata/articles/2016-01-01.json', 'r') as f:
    data = json.load(f)
df = pd.DataFrame(data)
el = df.iloc[0]
s = [df.iloc[i] for i in range(len(df))]
s

[apiUrl                https://content.guardianapis.com/us-news/2016/...
 fields                {'lastModified': '2017-07-14T20:17:15Z', 'shou...
 id                    us-news/2016/jan/01/tamir-rice-protest-clevela...
 isHosted                                                          False
 pillarId                                                    pillar/news
 pillarName                                                         News
 sectionId                                                       us-news
 sectionName                                                     US news
 type                                                            article
 webPublicationDate                                 2016-01-01T22:44:12Z
 webTitle              Tamir Rice protesters picket house of Clevelan...
 webUrl                https://www.theguardian.com/us-news/2016/jan/0...
 Name: 0, dtype: object,
 apiUrl                https://content.guardianapis.com/football/2016...
 fields                {'l

In [26]:
pub = el['webPublicationDate']
k = el['webUrl']
s = datetime.datetime.strptime(pub, "%Y-%m-%dT%H:%M:%SZ" )
s

datetime.datetime(2016, 1, 1, 22, 44, 12)

In [44]:
k = 'https://www.aaa.uni-augsburg.de/en/welcome_service/wohnungssuche/laenger_aufenthalte/'
try:
    r = requests.get(k.strip())
    r.raise_for_status()
    article = Article(k.strip())
    article.download()
    article.parse()
    print(article.text)
except HTTPError:
    print('***FAILED TO DOWNLOAD***', article.url)
    


***FAILED TO DOWNLOAD*** https://www.aaa.uni-augsburg.de/en/welcome_service/wohnungssuche/laenger_aufenthalte/


In [75]:
requests.get(k)

<Response [200]>

In [64]:
article.parse()

In [65]:
article.authors

['Rupert Neate']

In [85]:
article.publish_date

datetime.datetime(2016, 1, 31, 0, 0)

In [69]:
article.text

"New York lawyers have been secretly filmed advising how an African minister could use “grey money” to buy up a Manhattan brownstone, a private jet and a luxury yacht without US authorities – or his impoverished citizens back home – ever knowing.\n\nAn undercover activist posing as an adviser to an African mining minister filmed encounters with 13 US law firms, offering a rare glimpse into how US lawyers could possibly act to facilitate the flow of dirty money into the country despite Barack Obama repeatedly demanding a crackdown on global corruption.\n\nAlmost all the lawyers offered suggestions as to how the minister could move “grey money” and “suspect funds” into the US while ensuring his identity was never disclosed. None of them made suggestions that were illegal.\n\nOne lawyer, Mark Koplik, was recorded by an undercover investigator from anti-corruption charity Global Witness as saying: “So we have to scrub it at the beginning, if we can, or scrub it at the intermediary location

In [24]:
data = ["https://www.theguardian.com/society/2013/jan/01/domestic-abuse-changing-conversation",
"https://www.theguardian.com/money/2013/jan/01/merge-work-life-balance",
"https://www.theguardian.com/technology/blog/2013/jan/01/technology-links-newsbucket",
"https://www.theguardian.com/eyewitness-subscriber/picture/572",
"https://www.theguardian.com/eyewitness-subscriber/picture/573",
"https://www.theguardian.com/eyewitness-subscriber/picture/eyewitness-architecture7",
"https://www.theguardian.com/eyewitness-subscriber/picture/571",
"https://www.theguardian.com/world/2013/jan/01/fiscal-cliff-threat-house-republicans"]

In [25]:
data[0]

'https://www.theguardian.com/society/2013/jan/01/domestic-abuse-changing-conversation'